# Preparación del entorno: instalaciones y librerías

In [ ]:
#Instalaciones
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [107]:
#Librerías
import os
import numpy as np
import pandas as pd
import glob
import shutil
import torch
import torch.nn.functional as F
from torch import nn
from google.colab import drive
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, f1_score
from torch.optim import Adam
from sklearn.model_selection import ParameterGrid
from datasets import concatenate_datasets

# Dataset

In [ ]:
#Carga del dataset de España
dataset_es = load_dataset("alttal/SA_opiniones_indumentaria_ES_Espanya_splits")

In [ ]:
#Se eliminan columnas innecesarias
dataset_es = dataset_es.remove_columns(["id","__index_level_0__"])
dataset_es

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 2187
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 625
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 313
    })
})

# Métricas de evaluación

In [ ]:
#Definimos las métricas para evaluar el modelo
#Este conjunto de métricas depende de la tarea.
#Para clasificación de textos se suelen utilizar accuracy, precision, recall y F1-score.

def compute_metrics(pred):
  y_true = pred.label_ids              # son las etiquetas reales
  y_pred = pred.predictions.argmax(-1) # son las predicciones
  acc = accuracy_score(y_true, y_pred)
  precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

  #métricas por clase
  precision_per_class, recall_per_class, f1_per_class, _ = precision_recall_fscore_support(y_true, y_pred, average=None)

  #conversiones necesarias para evitar errores de registro en Tensorboard
  precision_per_class = list(precision_per_class)
  recall_per_class = list(recall_per_class)
  f1_per_class = list(f1_per_class)

  return {
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall,
      'precision_per_class': precision_per_class,
      'recall_per_class': recall_per_class,
      'f1_per_class': f1_per_class
  }

# Documentación del progreso

In [ ]:
#Montar unidad de disco
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Ruta raíz para guardar cada experimento
path = '/content/drive/My Drive/TFM/TFM-Experimentos'

#Cambiar directorio actual de trabajo
os.chdir(path)

#Verificar directorio actual
print("Directorio actual:", os.getcwd())

Directorio actual: /content/drive/My Drive/TFM/TFM-Experimentos


In [ ]:
#Definición de una función para eliminar logs antiguos (para evitar falta de almacenamiento)
def clean_old_logs(logging_dir, keep_last_n=1):
    log_files = sorted(glob.glob(os.path.join(logging_dir, '*')), key=os.path.getmtime)
    for log_file in log_files[:-keep_last_n]:
        os.remove(log_file)

In [ ]:
#Ruta al directorio de logs
log_dir = path

#Eliminar archivos antiguos de TensorBoard (para evitar acumulación de demasiados logs)
for file in os.listdir(log_dir):
    if file.startswith('events.out.tfevents'):
        os.remove(os.path.join(log_dir, file))

# Modelo 1

##Carga del modelo 1 y adaptación de los datos

In [ ]:
#Carga del modelo pre-entrenado
model2 = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest", num_labels=3)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#A modo de información, vemos la configuración del modelo
model2.config

RobertaConfig {
  "_name_or_path": "cardiffnlp/twitter-roberta-base-sentiment-latest",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "neutral",
    "2": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "neutral": 1,
    "positive": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.42.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [ ]:
#¿Cómo son las etiquetas del modelo? Importante porque deben ser iguales a las del dataset
model2.config.id2label

{0: 'negative', 1: 'neutral', 2: 'positive'}

In [ ]:
#Las etiquetas de los datasets son strings. Se las cambia a valores numéricos
def label_to_int(label):
    if label == "POS":
        return 2
    elif label == "NEG":
        return 0
    else:
        return 1

dataset_es_1 = dataset_es.map(lambda x: {"label": label_to_int(x["label"])})

##Tokenización del dataset

In [ ]:
#Carga del tokenizador específico
tokenizer2 = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

Consultamos cuál es la secuencia máxima que acepta el modelo

In [ ]:
#Sabemos que los modelos basados en BERT o RoBERTa sólo reciben unos 512 tokens como máximo. En este caso 514. Nos quedaremos con el valor 512.
print(f"La longitud máxima de secuencia aceptada por el modelo es: {model2.config.max_position_embeddings}")

La longitud máxima de secuencia aceptada por el modelo es: 514


Con esta información, se puede crear un tokenizador que se adapte mejor a los datos

In [ ]:
#Se define tokenizador del primer modelo
def tokenize_function2(example):
    return tokenizer2(example["text"], max_length=512, padding='max_length', truncation=True)

In [ ]:
#Tokenización del dataset
encoded_dataset_es_1 = dataset_es_1.map(tokenize_function2, batched=True)
#Luego de tokenizar se puede eliminar la columna "text" porque el modelo no la necesita
encoded_dataset_es_1 = encoded_dataset_es_1.remove_columns(["text"])
encoded_dataset_es_1

Map:   0%|          | 0/2187 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 2187
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 625
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 313
    })
})

In [ ]:
#Ejemplo de la tokenización en cada split
print(encoded_dataset_es_1['train'][0])
print(encoded_dataset_es_1['validation'][0])
print(encoded_dataset_es_1['test'][0])

{'label': 0, 'input_ids': [0, 9839, 162, 16954, 118, 542, 102, 843, 1192, 2714, 11163, 6764, 102, 162, 2677, 196, 260, 30249, 366, 117, 2677, 196, 260, 10, 8987, 6510, 2955, 897, 18105, 8647, 263, 3729, 5502, 13531, 3304, 260, 3304, 853, 1176, 19928, 366, 117, 3774, 13103, 2802, 493, 43045, 271, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

## Modificación de pesos

Al tener un dataset desbalanceado en el cual la mayoría de las instancias pertenecen a la clase POS, y con una minoría de instancias etiquetadas como NEU, se ha observado en otros experimentos que el modelo tiene grandes dificultades a la hora de predecir la clase neutral porque, al recibir tan pocos ejemplos, no parece poder aprenderla.

Una alternativa en estos casos es modificar los pesos y penalizar al modelo para que aprenda mejor a identificar la clase minoritaria.

In [ ]:
class_weights_dict = {}

#Iteración sobre cada split (train, test, validation)
for split in ['train', 'test', 'validation']:
    #Acceder a split
    dataset = encoded_dataset_es_1[split]

    #Convertir la columna 'label' a una serie de pandas
    labels_series = pd.Series(dataset['label'])

    #Calcular los pesos de clase
    class_weights = (1 - (labels_series.value_counts().sort_index() / len(labels_series))).values

    #Guardar los pesos en un diccionario
    class_weights_dict[split] = class_weights

print(class_weights_dict)

{'train': array([0.60265203, 0.85550983, 0.54183813]), 'test': array([0.6032, 0.856 , 0.5408]), 'validation': array([0.60063898, 0.85623003, 0.54313099])}


In [ ]:
#El trainer se basa en pytorch, por eso se modifica de array numpy a tensores pytorch
class_weights = torch.from_numpy(class_weights).float().to("cuda")
class_weights

tensor([0.6006, 0.8562, 0.5431], device='cuda:0')

In [ ]:
#Siguiendo la documentación, el trainer puede calcular la pérdida si se le provee un argumento 'labels'
#Fuente: https://huggingface.co/docs/transformers/main_classes/trainer & https://www.youtube.com/watch?v=u--UVvH-LIQ
#El que tenemos se llama 'label', por eso se lo modifica
encoded_dataset_es_1 = encoded_dataset_es_1.rename_column("label", "labels")

In [ ]:
#Creamos una clase que es subclase de la clase Trainer
#Desde ahí definimos cómo calcular la pérdida (loss function)
class WeightedLossTrainer(Trainer):
  def compute_metrics(self, model, inputs, return_outputs=False):
    #Introducir inputs al modelo
    outputs = model(**inputs)
    logits = outputs.get("logits")
    #Extraer labels
    labels = inputs.get("labels")
    #Definir función de pérdida con los nuevos pesos
    loss = loss_func = nn.CrossEntropyLoss(weight=class_weights)
    #Calcular pérdida
    loss = loss_func(logits, labels)
    return (loss, outputs) if return_outputs else loss

## Experimento 2.1.

In [ ]:
#Documentación
#Definición de directorios para el primer experimento
output_dir = "./outputs/weights_model2_lit"
shutil.make_archive("./outputs/weights_model2_lit", 'zip', output_dir)
logging_dir = "./logs/weights_model2_lit"
shutil.make_archive("./logs/weights_model2_lit", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

Entrenamiento con hiperparámetros recomendados

In [ ]:
#Definición de EarlyStopping
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,    # Número de épocas para esperar sin mejora
    early_stopping_threshold=0.0  # Umbral para la mejora mínima
)

In [ ]:
#Definición de hiperparámetros
training_args_weights_model2_lit = TrainingArguments(
    output_dir=output_dir,           # Directorio de salida
    logging_dir=logging_dir,         # Directorio para los logs
    eval_strategy="steps",           # Estrategia de evaluación
    save_strategy="steps",           # Estrategia de guardado (debe coincidir con la de evaluación)
    save_steps=1000,                 # Número de pasos entre cada guardado
    logging_strategy="steps",        # Estrategia de registro
    logging_steps=1000,              # Número de pasos entre cada registro
    num_train_epochs=3,              # Número de épocas de entrenamiento
    per_device_train_batch_size=16,  # Tamaño del batch de entrenamiento (podría ser también 32)
    per_device_eval_batch_size=16,   # Tamaño del batch de evaluación (podría ser también 32)
    learning_rate=3e-5,              # Tasa de aprendizaje
    weight_decay=0.01,               # Decadencia de peso (podría ser también 0.1)
    warmup_ratio=0.1,                # Proporción de épocas de calentamiento
    save_total_limit=1,              # Número máximo de checkpoints a guardar (para evitar problemas de almacenamiento)
    seed=42,                         # Semilla para garantizar reproducibilidad
    fp16=True,                       # Uso de fp16 para acelerar el entrenamiento
    load_best_model_at_end=True,     # Cargar el mejor modelo al final del entrenamiento
    metric_for_best_model="f1",      # Métrica objetivo, a optimizar para encontrar el mejor modelo
)

In [ ]:
#Definición de optimizador
optimizer = Adam(model2.parameters(), lr=3e-5, weight_decay=0.01)

In [ ]:
#Definición de un objeto de la clase Trainer para el primer experimento con el modelo 1
trainer_weights_model2_lit = WeightedLossTrainer(
    model = model2,
    args = training_args_weights_model2_lit,
    train_dataset = encoded_dataset_es_1['train'],
    eval_dataset = encoded_dataset_es_1['validation'],
    compute_metrics=compute_metrics,
    tokenizer = tokenizer2,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer2),
    optimizers = (optimizer, None),
    callbacks = [early_stopping])

Entrenamiento

In [ ]:
trainer_weights_model2_lit.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=411, training_loss=0.7364029710310219, metrics={'train_runtime': 54.756, 'train_samples_per_second': 119.823, 'train_steps_per_second': 7.506, 'total_flos': 1726287133713408.0, 'train_loss': 0.7364029710310219, 'epoch': 3.0})

Validación

In [ ]:
trainer_weights_model2_lit.evaluate()

Trainer is attempting to log a value of "[0.8015267175572519, 0.47058823529411764, 0.793939393939394]" of type <class 'list'> for key "eval/precision_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.84, 0.17777777777777778, 0.916083916083916]" of type <class 'list'> for key "eval/recall_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8203125, 0.25806451612903225, 0.8506493506493507]" of type <class 'list'> for key "eval/f1_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.5510281920433044,
 'eval_accuracy': 0.7795527156549521,
 'eval_f1': 0.753338092232152,
 'eval_precision': 0.7504812895087065,
 'eval_recall': 0.7795527156549521,
 'eval_precision_per_class': [0.8015267175572519,
  0.47058823529411764,
  0.793939393939394],
 'eval_recall_per_class': [0.84, 0.17777777777777778, 0.916083916083916],
 'eval_f1_per_class': [0.8203125, 0.25806451612903225, 0.8506493506493507],
 'eval_runtime': 0.9054,
 'eval_samples_per_second': 345.692,
 'eval_steps_per_second': 22.089,
 'epoch': 3.0}

Reporte de clasificación (sobre conjunto de validación)

In [ ]:
#Obtención de predicciones del conjunto de validación
predictions = trainer_weights_model2_lit.predict(encoded_dataset_es_1['validation'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.80      0.84      0.82       125
           1       0.47      0.18      0.26        45
           2       0.79      0.92      0.85       143

    accuracy                           0.78       313
   macro avg       0.69      0.64      0.64       313
weighted avg       0.75      0.78      0.75       313


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.80   0.84     0.82
    1      0.47   0.18     0.26
    2      0.79   0.92     0.85


In [ ]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

Test

Reporte de clasificación (sobre conjunto de test)

In [ ]:
#Obtención de predicciones del conjunto de test
predictions = trainer_weights_model2_lit.predict(encoded_dataset_es_1['test'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.78      0.90      0.83       248
           1       0.64      0.23      0.34        90
           2       0.81      0.86      0.83       287

    accuracy                           0.78       625
   macro avg       0.74      0.66      0.67       625
weighted avg       0.77      0.78      0.76       625


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.78   0.90     0.83
    1      0.64   0.23     0.34
    2      0.81   0.86     0.83


In [ ]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

## Grid Search

Búsqueda de hiperparámetros optimizados con la técnica Grid Search:

In [ ]:
#Definición del espacio de búsqueda
param_grid = {
    'num_train_epochs': [3, 10],
    'per_device_train_batch_size': [16, 32],
    'per_device_eval_batch_size': [16, 32],
    'learning_rate': [1e-5, 2e-5, 3e-5, 5e-5, 1e-4],
    'weight_decay': [0.01, 0.1],
    'warmup_ratio': [0.06, 0.1],
}

grid = ParameterGrid(param_grid)

In [ ]:
#Función para calcular el f1-score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {
        'f1': f1_score(labels, predictions, average='weighted')
    }

In [ ]:
#Inicialización de variables para guardar los mejores resultados
best_score = 0
best_params = None

In [ ]:
#Documentación
#Definición de directorios para el primer experimento
output_dir = "./outputs/weights_model2_search"
shutil.make_archive("./outputs/weights_model2_search", 'zip', output_dir)
logging_dir = "./logs/weights_model2_search"
shutil.make_archive("./logs/weights_model2_search", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

In [ ]:
#Bucle para probar los diferentes parámetros
for params in grid:
    print(f"Entrenando con parámetros: {params}")

    #Definición de hiperparámetros
    training_args_weights_model2_search = TrainingArguments(
        output_dir=output_dir,                                                  # Directorio de salida
        logging_dir=logging_dir,                                                # Directorio para los logs
        eval_strategy="steps",                                                  # Estrategia de evaluación
        save_strategy="steps",                                                  # Estrategia de guardado (debe coincidir con la de evaluación)
        save_steps=1000,                                                        # Número de pasos entre cada guardado
        logging_strategy="steps",                                               # Estrategia de registro
        logging_steps=1000,                                                     # Número de pasos entre cada registro
        num_train_epochs=params['num_train_epochs'],                            # Número de épocas de entrenamiento
        per_device_train_batch_size=params['per_device_train_batch_size'],      # Tamaño del batch de entrenamiento
        per_device_eval_batch_size=params['per_device_eval_batch_size'],        # Tamaño del batch de evaluación
        learning_rate=params['learning_rate'],                                  # Tasa de aprendizaje
        weight_decay=params['weight_decay'],                                    # Decadencia de peso
        warmup_ratio=params['warmup_ratio'],                                    # Proporción de épocas de calentamiento
        save_total_limit=1,                                                     # Número máximo de checkpoints a guardar (para evitar problemas de almacenamiento)
        seed=42,                                                                # Semilla para garantizar reproducibilidad
        fp16=True,                                                              # Uso de fp16 para acelerar el entrenamiento
        load_best_model_at_end=True,                                            # Cargar el mejor modelo al final del entrenamiento
        metric_for_best_model='f1',                                             # Métrica objetivo, a optimizar para encontrar el mejor modelo
    )

    #Definición de un objeto de la clase Trainer para la búsqueda con el modelo 1
    trainer_weights_model2_search = WeightedLossTrainer(
        model=model2,
        args=training_args_weights_model2_search,
        train_dataset=encoded_dataset_es_1['train'],
        eval_dataset=encoded_dataset_es_1['validation'],
        tokenizer=tokenizer2,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer2),
        compute_metrics=compute_metrics,
        optimizers = (optimizer, None),
        callbacks = [early_stopping]
    )

    #Entrenamiento
    trainer_weights_model2_search.train()

    #Limpiar logs antiguos después del entrenamiento en cada iteración
    clean_old_logs(logging_dir, keep_last_n=1)

    #Validación
    eval_result = trainer_weights_model2_search.evaluate()

    #Guardamos métrica objetivo
    score = eval_result.get('eval_f1', 0)

    #Siempre que la métrica objetivo sea mejor que el score guardado hasta el momento, se lo actualiza
    if score > best_score:
        best_score = score
        best_params = params

#Vemos los resultados del Grid Search
print(f"Mejores hiperparámetros: {best_params}")
print(f"Mejor puntuación F1: {best_score}")

Entrenando con parámetros: {'learning_rate': 1e-05, 'num_train_epochs': 3, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'warmup_ratio': 0.06, 'weight_decay': 0.01}


Step,Training Loss,Validation Loss


Entrenando con parámetros: {'learning_rate': 1e-05, 'num_train_epochs': 3, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'warmup_ratio': 0.06, 'weight_decay': 0.1}


Step,Training Loss,Validation Loss


Entrenando con parámetros: {'learning_rate': 1e-05, 'num_train_epochs': 3, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'warmup_ratio': 0.1, 'weight_decay': 0.01}


...

Entrenando con parámetros: {'learning_rate': 0.0001, 'num_train_epochs': 10, 'per_device_eval_batch_size': 32, 'per_device_train_batch_size': 32, 'warmup_ratio': 0.1, 'weight_decay': 0.01}


Step,Training Loss,Validation Loss


Entrenando con parámetros: {'learning_rate': 0.0001, 'num_train_epochs': 10, 'per_device_eval_batch_size': 32, 'per_device_train_batch_size': 32, 'warmup_ratio': 0.1, 'weight_decay': 0.1}


Step,Training Loss,Validation Loss


Mejores hiperparámetros: {'learning_rate': 1e-05, 'num_train_epochs': 3, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'warmup_ratio': 0.1, 'weight_decay': 0.01}
Mejor puntuación F1: 0.8078480471514033


## Experimento 2.2.

In [ ]:
#Documentación
#Definición de directorios para el segundo experimento
output_dir = "./outputs/weights_model2_opt"
shutil.make_archive("./outputs/weights_model2_opt", 'zip', output_dir)
logging_dir = "./logs/weights_model2_opt"
shutil.make_archive("./logs/weights_model2_opt", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

Entrenamiento con hiperparámetros optimizados

In [ ]:
#Definición de EarlyStopping
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,    # Número de épocas para esperar sin mejora
    early_stopping_threshold=0.0  # Umbral para la mejora mínima
)

Mejores hiperparámetros: {'learning_rate': 1e-05, 'num_train_epochs': 3, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'warmup_ratio': 0.1, 'weight_decay': 0.01}
Mejor puntuación F1: 0.8078480471514033

In [89]:
#Definición de hiperparámetros (basados en Grid Search)
training_args_weights_model2_opt = TrainingArguments(
    output_dir=output_dir,                                # Directorio de salida
    logging_dir=logging_dir,                              # Directorio para los logs
    eval_strategy="steps",                                # Estrategia de evaluación
    save_strategy="steps",                                # Estrategia de guardado (debe coincidir con la de evaluación)
    save_steps=1000,                                      # Número de pasos entre cada guardado
    logging_strategy="steps",                             # Estrategia de registro
    logging_steps=1000,                                   # Número de pasos entre cada registro
    num_train_epochs=3,                                   # Número de épocas de entrenamiento
    per_device_train_batch_size=16,                       # Tamaño del batch de entrenamiento
    per_device_eval_batch_size=16,                        # Tamaño del batch de evaluación
    learning_rate=1e-05,                                  # Tasa de aprendizaje
    weight_decay=0.1,                                    # Decadencia de peso
    warmup_ratio=0.01,                                    # Proporción de épocas de calentamiento
    save_total_limit=1,                                   # Número máximo de checkpoints a guardar (para evitar problemas de almacenamiento)
    seed=42,                                              # Semilla para garantizar reproducibilidad
    fp16=True,                                            # Uso de fp16 para acelerar el entrenamiento
    load_best_model_at_end=True,                          # Cargar el mejor modelo al final del entrenamiento
    metric_for_best_model="f1",                           # Métrica objetivo, a optimizar para encontrar el mejor modelo
)

In [90]:
#Definición de optimizador
optimizer = Adam(model2.parameters(), lr=params['learning_rate'], weight_decay=params['weight_decay'])

In [91]:
#Definición de un objeto de la clase Trainer para el segundo experimento con el modelo 1
trainer_weights_model2_opt = WeightedLossTrainer(
    model = model2,
    args = training_args_weights_model2_opt,
    train_dataset = encoded_dataset_es_1['train'],
    eval_dataset = encoded_dataset_es_1['validation'],
    compute_metrics=compute_metrics,
    tokenizer = tokenizer2,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer2),
    optimizers = (optimizer, None),
    callbacks = [early_stopping])

Entrenamiento

In [92]:
trainer_weights_model2_opt.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=411, training_loss=0.5136172997690466, metrics={'train_runtime': 57.7744, 'train_samples_per_second': 113.562, 'train_steps_per_second': 7.114, 'total_flos': 1726287133713408.0, 'train_loss': 0.5136172997690466, 'epoch': 3.0})

Validación

In [93]:
trainer_weights_model2_opt.evaluate()

{'eval_loss': 0.6518620848655701,
 'eval_f1': 0.7014412627528296,
 'eval_runtime': 0.9256,
 'eval_samples_per_second': 338.165,
 'eval_steps_per_second': 21.608,
 'epoch': 3.0}

Reporte de clasificación (sobre conjunto de validación)

In [94]:
#Obtención de predicciones del conjunto de validación
predictions = trainer_weights_model2_opt.predict(encoded_dataset_es_1['validation'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.76      0.87      0.81       125
           1       0.00      0.00      0.00        45
           2       0.76      0.90      0.83       143

    accuracy                           0.76       313
   macro avg       0.51      0.59      0.55       313
weighted avg       0.65      0.76      0.70       313


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.76   0.87     0.81
    1      0.00   0.00     0.00
    2      0.76   0.90     0.83


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [95]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

Test

Reporte de clasificación (sobre conjunto de test)

In [96]:
#Obtención de predicciones del conjunto de test
predictions = trainer_weights_model2_opt.predict(encoded_dataset_es_1['test'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.70      0.82      0.76       248
           1       0.00      0.00      0.00        90
           2       0.72      0.84      0.77       287

    accuracy                           0.71       625
   macro avg       0.47      0.55      0.51       625
weighted avg       0.61      0.71      0.66       625


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.70   0.82     0.76
    1      0.00   0.00     0.00
    2      0.72   0.84     0.77


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [97]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

## Experimento extra 2.3.

In [98]:
#Documentación
#Definición de directorios para el tercer experimento
output_dir = "./outputs/weights_model2_est"
shutil.make_archive("./outputs/weights_model2_est", 'zip', output_dir)
logging_dir = "./logs/weights_model2_est"
shutil.make_archive("./logs/weights_model2_est", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

Entrenamiento con hiperparámetros originales

In [99]:
#Definición de hiperparámetros estándar
training_args_weights_model2_est = TrainingArguments(
    output_dir=output_dir,
    logging_dir=logging_dir,
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=1000,
    logging_strategy="steps",
    logging_steps=1000,
    seed=42,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

In [100]:
#Definición de un objeto de la clase Trainer para el tercer experimento con el modelo 1
trainer_weights_model2_est = WeightedLossTrainer(
    model = model2,
    args = training_args_weights_model2_est,
    train_dataset = encoded_dataset_es_1['train'],
    eval_dataset = encoded_dataset_es_1['validation'],
    compute_metrics=compute_metrics,
    tokenizer = tokenizer2,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer2),
    optimizers = (optimizer, None),
    callbacks = [early_stopping])

Entrenamiento

In [101]:
trainer_weights_model2_est.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=822, training_loss=0.5467456158640321, metrics={'train_runtime': 73.6788, 'train_samples_per_second': 89.049, 'train_steps_per_second': 11.157, 'total_flos': 1726287133713408.0, 'train_loss': 0.5467456158640321, 'epoch': 3.0})

Validación

In [102]:
trainer_weights_model2_est.evaluate()

{'eval_loss': 0.6620157957077026,
 'eval_f1': 0.7044155812531556,
 'eval_runtime': 1.0848,
 'eval_samples_per_second': 288.541,
 'eval_steps_per_second': 36.874,
 'epoch': 3.0}

Reporte de clasificación (sobre conjunto de validación)

In [103]:
#Obtención de predicciones del conjunto de validación
predictions = trainer_weights_model2_est.predict(encoded_dataset_es_1['validation'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.76      0.88      0.81       125
           1       0.00      0.00      0.00        45
           2       0.77      0.90      0.83       143

    accuracy                           0.76       313
   macro avg       0.51      0.59      0.55       313
weighted avg       0.65      0.76      0.70       313


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.76   0.88     0.81
    1      0.00   0.00     0.00
    2      0.77   0.90     0.83


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [104]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

Test

Reporte de clasificación (sobre conjunto de test)

In [105]:
#Obtención de predicciones del conjunto de test
predictions = trainer_weights_model2_est.predict(encoded_dataset_es_1['test'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.70      0.81      0.75       248
           1       0.00      0.00      0.00        90
           2       0.71      0.85      0.77       287

    accuracy                           0.71       625
   macro avg       0.47      0.55      0.51       625
weighted avg       0.61      0.71      0.65       625


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.70   0.81     0.75
    1      0.00   0.00     0.00
    2      0.71   0.85     0.77


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [106]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

# Mejor modelo, nuevo dataset

## Dataset

In [108]:
#Carga del dataset de España
dataset_ar = load_dataset("alttal/SA_opiniones_indumentaria_ES_Argentina_splits")
dataset_ar

Generating train split:   0%|          | 0/2187 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/625 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/313 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'text', '__index_level_0__'],
        num_rows: 2187
    })
    test: Dataset({
        features: ['id', 'label', 'text', '__index_level_0__'],
        num_rows: 625
    })
    validation: Dataset({
        features: ['id', 'label', 'text', '__index_level_0__'],
        num_rows: 313
    })
})

In [109]:
#Unimos los splits: sólo necesitaremos uno para el test
dataset_ar = concatenate_datasets([dataset_ar['train'], dataset_ar['test'], dataset_ar['validation']])

#Ejemplo
print(dataset_ar)

Dataset({
    features: ['id', 'label', 'text', '__index_level_0__'],
    num_rows: 3125
})


In [110]:
#Se eliminan columnas innecesarias
dataset_ar = dataset_ar.remove_columns(["id","__index_level_0__"])
dataset_ar

Dataset({
    features: ['label', 'text'],
    num_rows: 3125
})

Adaptación de los datos

In [111]:
#Las etiquetas de los datasets son strings. Se las cambia a valores numéricos
def label_to_int(label):
    if label == "POS":
        return 2
    elif label == "NEG":
        return 0
    else:
        return 1

dataset_ar_1 = dataset_ar.map(lambda x: {"label": label_to_int(x["label"])})

Map:   0%|          | 0/3125 [00:00<?, ? examples/s]

## Tokenización

In [113]:
#Tokenización del dataset
encoded_dataset_ar_1 = dataset_ar_1.map(tokenize_function2, batched=True)
#Luego de tokenizar se puede eliminar la columna "text" porque el modelo no la necesita
encoded_dataset_ar_1 = encoded_dataset_ar_1.remove_columns(["text"])
encoded_dataset_ar_1

Map:   0%|          | 0/3125 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 3125
})

## Experimento con dataset de Argentina

In [114]:
#Obtención de predicciones del conjunto de test
predictions = trainer_weights_model2_lit.predict(encoded_dataset_ar_1)
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.50      0.81      0.62       703
           1       0.00      0.00      0.00       333
           2       0.85      0.82      0.83      2089

    accuracy                           0.73      3125
   macro avg       0.45      0.54      0.49      3125
weighted avg       0.68      0.73      0.70      3125


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.50   0.81     0.62
    1      0.00   0.00     0.00
    2      0.85   0.82     0.83


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif